### imports 

In [1]:
import os
from os import path
import configparser
from github import Github
import pandas as pd
import numpy
import datetime
from datetime import datetime
from datetime import timezone
import requests
from requests.adapters import HTTPAdapter, Retry

In [2]:
import githubanalysis.processing.repo_name_clean as name_clean
import githubanalysis.processing.get_repo_connection as ghconnect
import githubanalysis.processing.setup_github_auth as ghauth
import githubanalysis.processing.get_all_pages_issues as getallissues
import githubanalysis.analysis.calc_days_since_repo_creation as dayssince
import githubanalysis.processing.summarise_repo_stats
from githubanalysis.processing.summarise_repo_stats import RepoStatsSummariser
import utilities.get_default_logger as loggit
import utilities.chunker as chunker

### access token 

In [3]:
# set up github access token with github package: 

config = configparser.ConfigParser()
config.read('../config.cfg')
config.sections()

access_token = config['ACCESS']['token']
g = Github(access_token) 

### get stats for 1 repo  

In [4]:
config_path = '../config.cfg'
#config_path = '../../githubanalysis/config.cfg'

In [5]:

logger = loggit.get_default_logger(console=True, set_level_to='DEBUG', log_name='../../logs/summarise_repo_stats_NOTEBOOK_logs.txt')  
repo_summariser = RepoStatsSummariser(logger)


repo_summariser.summarise_repo_stats(repo_name='dacapobench/dacapobench', config_path='../../githubanalysis/config.cfg', per_pg=1, verbose=True)

GH link opened
API response for getting total commits: <Response [200]>
API response for getting total commits in year: <Response [200]>
API response for getting PRs info: <Response [200]>
Stats for dacapobench/dacapobench: {'repo_name': 'dacapobench/dacapobench', 'issues_enabled': True, 'devs': 25, 'total_commits': 1686, 'total_commits_last_year': 176, 'has_PRs': True, 'last_PR_update': datetime.datetime(2023, 11, 17, 3, 38, 51, tzinfo=datetime.timezone.utc), 'closed_tickets': 171, 'repo_age_days': 2253, 'repo_license': License(name="Apache License 2.0"), 'repo_visibility': True, 'repo_language': dict_keys(['Java', 'XSLT', 'Python', 'Perl', 'Shell', 'Batchfile', 'HTML', 'CSS'])}


{'repo_name': 'dacapobench/dacapobench',
 'issues_enabled': True,
 'devs': 25,
 'total_commits': 1686,
 'total_commits_last_year': 176,
 'has_PRs': True,
 'last_PR_update': datetime.datetime(2023, 11, 17, 3, 38, 51, tzinfo=datetime.timezone.utc),
 'closed_tickets': 171,
 'repo_age_days': 2253,
 'repo_license': License(name="Apache License 2.0"),
 'repo_visibility': True,
 'repo_language': dict_keys(['Java', 'XSLT', 'Python', 'Perl', 'Shell', 'Batchfile', 'HTML', 'CSS'])}

In [6]:
gh_urls_df = pd.read_csv("../../data/todo_2024-02-06.csv", header=0, usecols=['ZenodoID', 'repo_name'], dtype={'ZenodoID':'Int64', 'repo_name':'str'})
gh_urls_df.shape

(1062, 2)

In [7]:
gh_urls_df = gh_urls_df[:10]
print(gh_urls_df)

   ZenodoID                                          repo_name
0     32712                              harnesscloud/irm-nova
1   5899728  ericteboul/Fu-Teboul-et-al.-2022.-Gq-activatio...
2   3608671            blu-base/tesla-pump-design-optimization
3   1122777                          gcube-team/gcube-releases
4   6787506                          IPCC-WG1/Chapter-10_Fig13
5   5545932                                veronicamorfi/ml4bl
6   5608598                       zackspica/Gridded-SlantStack
7   6079286                     shaylematsuda/Mcap_Larval_Temp
8   3266016                      LXHien88/Flood_forecasting_HB
9   4683096            elvisburchia/DigitalSoilMappingSoftware


In [8]:
for repo in gh_urls_df.repo_name:
    print(f">>> Getting data for repo {repo}")
    repo_summariser.summarise_repo_stats(repo_name=repo, config_path='../../githubanalysis/config.cfg', per_pg=1, verbose=True)

>>> Getting data for repo harnesscloud/irm-nova
GH link opened
API response for getting total commits: <Response [200]>
API response for getting total commits in year: <Response [200]>
API response for getting PRs info: <Response [200]>


ERROR:Error in checking closed issue numbers at harnesscloud/irm-nova and config path ../../githubanalysis/config.cfg: 'created_at'.


column [] not present in all_issues df; 


ERROR:Error in checking license of repo at harnesscloud/irm-nova with config path ../../githubanalysis/config.cfg: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/licenses/licenses#get-the-license-for-a-repository"}.


key(s) [] are missing; 
Stats for harnesscloud/irm-nova: {'repo_name': 'harnesscloud/irm-nova', 'issues_enabled': True, 'devs': 6, 'total_commits': 119, 'total_commits_last_year': 0, 'has_PRs': False, 'last_PR_update': None, 'repo_age_days': 3241, 'repo_visibility': True, 'repo_language': dict_keys(['Python', 'Shell'])}
>>> Getting data for repo ericteboul/Fu-Teboul-et-al.-2022.-Gq-activation-in-BLA-parvalbumin-interneurons-Nature-Communications-2022
GH link opened
API response for getting total commits: <Response [200]>
API response for getting total commits in year: <Response [200]>
API response for getting PRs info: <Response [200]>


...

API response total_commits_last_year fail exception: 'last'
<class 'KeyError'>
'last'
API response for getting total commits in year: <Response [403]>


ERROR:Error in checking closed issue numbers at elvisburchia/DigitalSoilMappingSoftware and config path ../../githubanalysis/config.cfg: 'created_at'.


column [] not present in all_issues df; 


ERROR:Error in checking license of repo at elvisburchia/DigitalSoilMappingSoftware with config path ../../githubanalysis/config.cfg: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/licenses/licenses#get-the-license-for-a-repository"}.


key(s) [] are missing; 
Stats for elvisburchia/DigitalSoilMappingSoftware: {'repo_name': 'elvisburchia/DigitalSoilMappingSoftware', 'issues_enabled': True, 'devs': None, 'total_commits': None, 'repo_age_days': 1029, 'repo_visibility': True, 'repo_language': dict_keys(['R'])}
